<a href="https://colab.research.google.com/github/Youngseo-Jeon0313/computer_vision/blob/main/1_YOLO_%EB%AA%A8%EB%8D%B8_%EC%9D%B4%EC%9A%A9%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5


In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-218-g7539cd7 torch 1.10.0+cu111 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.0/107.7 GB disk)


In [ ]:
!apt install fonts-namu -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

#한글 폰트 설정하기
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package fonts-namu


generated new fontManager


### 1. 학습이미지 수집
  수집한 이미지를 저장하기 위한 폴더를 생성하고 필요한 함수를 정의해야 한다.

  #os.path와 관련된 다양한 조작 코드 참조
  https://docs.python.org/ko/3/library/os.path.html

  #shutil-고수준 파일 연산 관련 파일
  https://docs.python.org/ko/3/library/shutil.html

In [37]:
import os
import shutil

directory_list = [
    './custom_dataset/train/',
    './custom_dataset/test/',
]

# 초기 디렉토리 만들기
for directory in directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)

# 수집한 이미지를 학습 데이터와 평가 데이터로 구분하는 함수
def dataset_split(query, train_cnt):
    # 학습 및 평가 데이터셋 디렉토리 만들기
    for directory in directory_list:
        if not os.path.isdir(directory + '/' + query):
            os.makedirs(directory + '/' + query)
    # 학습 및 평가 데이터셋 준비하기
    cnt = 0
    for file_name in os.listdir(query):
        if cnt < train_cnt:
            print(f'[Train Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/train/' + query + '/' + file_name)
        else:
            print(f'[Test Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/test/' + query + '/' + file_name)
        cnt += 1
    shutil.rmtree(query)

##이미지 가져오기


###우리가 가진 data의 내용 확인해보기
일단 보니까 bounding box가 여러개 있고, 그에 따른 category_id가 정해져있음을 알 수 있음

In [22]:
%cat /content/dataset/labels/livestock_cow_bbox_000001.txt

1 0.246354 0.496296 0.338934 0.627318
1 0.825000 0.468519 0.882383 0.626418
1 0.881771 0.506481 0.983395 0.735642
1 0.520833 0.398148 0.577854 0.512594


In [23]:
%cat /content/dataset/livestock_data.yaml

names:
- pig
- cow
nc: 2
train: /content/dataset/train.txt
val: /content/dataset/val.txt


In [24]:
%cd /
#glob을 이용해서 dir에 있는 * 또는 ? 파일들을 불러오는 역할을 함
from glob import glob 
img_list= glob('/content/dataset/images/livestock_*.jpg')

print(len(img_list))

/
200


###배열 또는 행렬을 무작위 학습 및 테스트 하위 집합으로 분할시키는 코드이다. 참고!!

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [25]:
#이제 트레이닝 세트와 테스트셋으로 나눕니다.
from sklearn.model_selection import train_test_split 

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=20)

print(len(train_img_list), len(val_img_list))

160 40


In [26]:
with open('/content/dataset/train.txt','w') as f:
  f.write('\n'.join(train_img_list)+'\n')

with open('/content/dataset/val.txt','w') as f:
  f.write('\n'.join(val_img_list)+'\n')



#자 이렇게 하면 나눠졌어여!!


data yaml파일에는 클래스 이름들과 클래스 개수 그리고 train과 valid set의 주소가 적혀있다. ..여기에 이걸 잘 넣어주어야 함!

In [27]:
import yaml

with open('/content/dataset/data.yaml','r') as f:
  data=yaml.load(f)

print(data)
data['train']='/content/dataset/train.txt'
data['val']='/content/dataset/val.txt'

with open('/content/dataset/livestock_data.yaml', 'w') as f:
  yaml.dump(data,f)

print(data)

{'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt', 'nc': 2, 'names': ['pig', 'cow']}
{'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt', 'nc': 2, 'names': ['pig', 'cow']}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [28]:
import json

for i in range(1,101):
    num = str(i).zfill(4)
    with open('/content/dataset/images/livestock_cow_bbox_00' + num + '.json', 'r') as f:
        json_data = json.load(f)
        label_info = json_data['label_info']

        image_name = label_info['image']['file_name'] # 이미지 파일명
        image_width = label_info['image']['width'] # 이미지 넓이
        image_height = label_info['image']['height'] # 이미지 높이
        annotations =  label_info['annotations']
        image_name = image_name.replace(".jpg", "")
        file = open("/content/dataset/labels/"+image_name + ".txt", "w")
        for annot in annotations:
            x1, y1, x2, y2 = annot['bbox'] # 객체 1개의 bbox 위치 좌표
            x1 = x1/image_width
            y1 = y1/image_height
            x2 = (x2-x1)/image_width
            y2 = (y2-y1)/image_height
            file.write(f"1 {x1:f} {y1:f} {x2:f} {y2:f}\n")
        file.close()


In [20]:
import json

for i in range(1,101):
    num = str(i).zfill(4)
    with open('/content/dataset/images/livestock_pig_bbox_00' + num + '.json', 'r') as f:
        json_data = json.load(f)
        label_info = json_data['label_info']

        image_name = label_info['image']['file_name'] # 이미지 파일명
        image_width = label_info['image']['width'] # 이미지 넓이
        image_height = label_info['image']['height'] # 이미지 높이
        annotations =  label_info['annotations']
        image_name = image_name.replace(".jpg", "")
        file = open("/content/dataset/labels/"+image_name + ".txt", "w")
        for annot in annotations:
            x1, y1, x2, y2 = annot['bbox'] # 객체 1개의 bbox 위치 좌표
            x1 = x1/image_width
            y1 = y1/image_height
            x2 = (x2-x1)/image_width
            y2 = (y2-y1)/image_height
            file.write(f"0 {x1:f} {y1:f} {x2:f} {y2:f}\n")
        file.close()

In [ ]:
%cd /content/yolov5/


!python train.py --img 40 --batch 16 --epochs 10 --data /content/dataset/livestock_data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolov5_results


/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/dataset/livestock_data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=40, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-218-g7539cd7 torch 1.10.0+cu111 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hs

In [14]:

from utils.plots import plot_results
plot_results('path/to/results.csv') 

AssertionError: ignored